In [2]:
import os

In [3]:
%pwd   #currently i am inside research folder

'c:\\Users\\Prarthana\\Desktop\\projects\\kidney\\kidney_disease_classification\\research'

In [4]:
os.chdir("../")  #move out from research folder

In [5]:
%pwd

'c:\\Users\\Prarthana\\Desktop\\projects\\kidney\\kidney_disease_classification'

In [6]:
from dataclasses import dataclass
from pathlib import Path

In [7]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
from src.cnnClassifier.constants import *

In [9]:
import joblib

In [10]:
from cnnClassifier.utils.common import read_yaml, create_directories

In [11]:
from pathlib import Path

In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [13]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [23]:
class DataIngestion:
    def __init__(self, config):
        self.config = config
        
    def download_file(self) -> str:
        """Download file from the given URL"""
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs(os.path.dirname(zip_download_dir), exist_ok=True)

            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            # Extract file ID correctly
            file_id = dataset_url.split("/")[-2]
            download_url = f"https://drive.google.com/uc?id={file_id}"

            # Download file using gdown
            gdown.download(download_url, zip_download_dir, quiet=False)

            logger.info(f"File downloaded successfully: {zip_download_dir}")
        except Exception as e:
            logger.error(f"Error downloading file: {e}")
            return str(e)

    def extract_zip_file(self):
        """Extracts the zip file into the data directory"""
        try:
            unzip_path = self.config.unzip_dir
            os.makedirs(unzip_path, exist_ok=True)

            if not os.path.exists(self.config.local_data_file):
                raise FileNotFoundError(f"Zip file not found: {self.config.local_data_file}")

            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)

            logger.info(f"File extracted successfully to {unzip_path}")

        except Exception as e:
            logger.error(f"Error extracting file: {e}")
            raise e


In [24]:
%pwd

'c:\\Users\\Prarthana\\Desktop\\projects\\kidney\\kidney_disease_classification'

In [26]:
import os
print(os.path.exists("artifacts/data_ingestion/data.zip"))

False


In [27]:
#define pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-04-01 23:09:28,819: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-01 23:09:28,824: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-01 23:09:28,827: INFO: common: created directory at: artifacts]
[2025-04-01 23:09:28,830: INFO: common: created directory at: data_ingestion]
[2025-04-01 23:09:28,834: INFO: 1626317731: Downloading data from https://drive.google.com/file/d/1e3q72lfBRd5KX8KulVX39q34vW3y2M2b/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?id=1e3q72lfBRd5KX8KulVX39q34vW3y2M2b
From (redirected): https://drive.google.com/uc?id=1e3q72lfBRd5KX8KulVX39q34vW3y2M2b&confirm=t&uuid=ff0a8214-9b02-468a-adda-4ccf732c8b72
To: c:\Users\Prarthana\Desktop\projects\kidney\kidney_disease_classification\artifacts\data_ingestion\data.zip
100%|██████████| 55.8M/55.8M [34:51<00:00, 26.7kB/s]  

[2025-04-01 23:44:26,140: INFO: 1626317731: File downloaded successfully: artifacts/data_ingestion/data.zip]


[2025-04-01 23:44:27,956: INFO: 1626317731: File extracted successfully to artifacts/data_ingestion]
